In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale if not already
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

train_data_path = 'double_mnist/train'
val_data_path = 'double_mnist/val'
test_data_path = 'double_mnist/test'

# Create ImageFolder datasets for training, validation, and test sets
train_dataset = ImageFolder(train_data_path, transform=transform)
val_dataset = ImageFolder(val_data_path, transform=transform)
test_dataset = ImageFolder(test_data_path, transform=transform)

class_to_label = {str(i).zfill(2): i for i in range(100)}
# Define batch sizes
batch_size = 64

# Create DataLoader instances for training, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Number of classes
num_classes = 100

class_names = train_dataset.classes
class_names = val_dataset.classes

print(class_names)






['03', '07', '10', '22', '27', '34', '39', '40', '48', '52', '58', '61', '64', '71', '93', '99']


In [3]:
class SimpleCNN(nn.Module):
    def __init__(self, input_channels, num_classes, kernel_size=3, pool_size=2, stride=2, dropout_rate=0.5):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 32, kernel_size=kernel_size, padding=int((kernel_size - 1) / 2))
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=pool_size, stride=stride)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=int((kernel_size - 1) / 2))
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=pool_size, stride=stride)
        var1 = int((28 - pool_size) / stride) + 1
        self.dim = int((var1 - pool_size) / stride) + 1
        self.fc1 = nn.Linear(64 * self.dim * self.dim, 128)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * self.dim * self.dim)
        x = self.relu3(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = SimpleCNN(input_channels=1, num_classes=num_classes)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [5]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        class_names = train_dataset.classes
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += 2*labels.size(0)
        for i in range(labels.size(0)):
            if(predicted[i]%10 == labels[i]%10):
                correct_train += 1
            if((predicted[i]//10)%10 == (labels[i]//10)%10):
                correct_train += 1

    train_accuracy = 100 * correct_train / total_train
    average_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {average_train_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%')


Epoch 1/5 - Train Loss: 1.1299 - Train Accuracy: 69.41%
Epoch 2/5 - Train Loss: 0.9388 - Train Accuracy: 74.61%
Epoch 3/5 - Train Loss: 0.7977 - Train Accuracy: 78.40%
Epoch 4/5 - Train Loss: 0.7072 - Train Accuracy: 80.72%
Epoch 5/5 - Train Loss: 0.6404 - Train Accuracy: 82.45%


In [ ]:
# Define the number of epochs
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    correct_train = 0
    total_train = 0
    
    # Training loop
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        
        # Compute digit-wise accuracy for training set
        _, predicted = torch.max(outputs.data, 1)
        total_train += 2 * labels.size(0)
        for i in range(labels.size(0)):
            if (predicted[i] % 10 == labels[i] % 10) and ((predicted[i] // 10) % 10 == (labels[i] // 10) % 10):
                correct_train += 2
            elif (predicted[i] % 10 == labels[i] % 10) or ((predicted[i] // 10) % 10 == (labels[i] // 10) % 10):
                correct_train += 1

    train_accuracy = 100 * correct_train / total_train
    average_train_loss = total_train_loss / len(train_loader)

    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {average_train_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%')

    # Validation loop
    model.eval()
    total_val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_val_loss += loss.item()
            
            # Compute digit-wise accuracy for validation set
            _, predicted = torch.max(outputs.data, 1)
            total_val += 2 * labels.size(0)
            for i in range(labels.size(0)):
                if (predicted[i] % 10 == labels[i] % 10) and ((predicted[i] // 10) % 10 == (labels[i] // 10) % 10):
                    correct_val += 2
                elif (predicted[i] % 10 == labels[i] % 10) or ((predicted[i] // 10) % 10 == (labels[i] // 10) % 10):
                    correct_val += 1

    val_accuracy = 100 * correct_val / total_val
    average_val_loss = total_val_loss / len(val_loader)

    print(f'Epoch {epoch+1}/{num_epochs} - Validation Loss: {average_val_loss:.4f} - Validation Accuracy: {val_accuracy:.2f}%')

# At the end of training, you will have both training and validation loss and accuracy.


Epoch 1/10 - Train Loss: 2.5595 - Train Accuracy: 38.83%
Epoch 1/10 - Validation Loss: 11.8090 - Validation Accuracy: 10.84%
Epoch 2/10 - Train Loss: 1.2117 - Train Accuracy: 68.29%
Epoch 2/10 - Validation Loss: 14.9568 - Validation Accuracy: 9.60%
Epoch 3/10 - Train Loss: 0.9217 - Train Accuracy: 75.66%
Epoch 3/10 - Validation Loss: 17.6873 - Validation Accuracy: 9.68%
Epoch 4/10 - Train Loss: 0.7822 - Train Accuracy: 79.21%
Epoch 4/10 - Validation Loss: 19.5631 - Validation Accuracy: 9.56%
Epoch 5/10 - Train Loss: 0.6870 - Train Accuracy: 81.67%
Epoch 5/10 - Validation Loss: 20.1995 - Validation Accuracy: 9.63%
Epoch 6/10 - Train Loss: 0.6274 - Train Accuracy: 83.37%
Epoch 6/10 - Validation Loss: 23.4444 - Validation Accuracy: 8.50%
Epoch 7/10 - Train Loss: 0.5704 - Train Accuracy: 84.84%
Epoch 7/10 - Validation Loss: 22.4632 - Validation Accuracy: 11.00%
Epoch 8/10 - Train Loss: 0.5290 - Train Accuracy: 85.72%
Epoch 8/10 - Validation Loss: 24.9378 - Validation Accuracy: 9.10%
Epoch 

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    correct_train = 0
    total_train = 0
    
    for images, labels in val_loader:
        class_names = val_dataset.classes
        # print(class_names)
        # print(labels)
        optimizer.zero_grad()
        outputs = model(images)
        # print(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        # print(predicted)
        total_train += 2*labels.size(0)
        for i in range(labels.size(0)):
            if(predicted[i]%10 == labels[i]%10):
                correct_train += 1
            if((predicted[i]//10)%10 == (labels[i]//10)%10):
                correct_train += 1
        # correct_train += (predicted == labels).sum().item()

    train_accuracy = 100 * correct_train / total_train
    average_train_loss = total_train_loss / len(val_loader)

    # Evaluate on validation set
    # model.eval()
    # correct_val = 0
    # total_val = 0
    # total_val_loss = 0
    
    # with torch.no_grad():
    #     for images, labels in val_loader:
    #         outputs = model(images)
    #         loss = criterion(outputs, labels)
    #         total_val_loss += loss.item()
    #         _, predicted = torch.max(outputs.data, 1)
    #         # print(labels)
    #         # print(predicted)
    #         total_val += labels.size(0)
    #         correct_val += (predicted == labels).sum().item()
    
    # validation_accuracy = 100 * correct_val / total_val
    # average_val_loss = total_val_loss / len(val_loader)
    # print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {average_train_loss:.4f} - Train Accuracy: {train_accuracy:.2f}% - Validation Loss: {average_val_loss:.4f} - Validation Accuracy: {validation_accuracy:.2f}%')
    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {average_train_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%')

# Evaluate on test set
# model.eval()
# correct = 0
# total = 0

# with torch.no_grad():
#     for images, labels in test_loader:
#         outputs = model(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# test_accuracy = 100 * correct / total

# print(f'Test Accuracy: {test_accuracy:.2f}%')


Epoch 1/10 - Train Loss: 4.7381 - Train Accuracy: 39.98%
Epoch 2/10 - Train Loss: 3.5479 - Train Accuracy: 45.15%
Epoch 3/10 - Train Loss: 3.0284 - Train Accuracy: 40.52%
Epoch 4/10 - Train Loss: 2.3435 - Train Accuracy: 53.75%
Epoch 5/10 - Train Loss: 2.0733 - Train Accuracy: 54.16%
Epoch 6/10 - Train Loss: 1.9289 - Train Accuracy: 53.54%
Epoch 7/10 - Train Loss: 1.8004 - Train Accuracy: 56.84%
Epoch 8/10 - Train Loss: 1.8493 - Train Accuracy: 58.89%
Epoch 9/10 - Train Loss: 2.1813 - Train Accuracy: 50.19%
Epoch 10/10 - Train Loss: 2.1456 - Train Accuracy: 48.74%
